### Imports

In [1]:
from simcse import SimCSE
import pandas as pd
import numpy as np
from pathlib import Path

### Load model and dataset

In [2]:
model = SimCSE('./data/dental_model/')

Some weights of BertModel were not initialized from the model checkpoint at ./data/dental_model/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
dataset = pd.read_csv('data/input/cluster_simcse_input.csv', index_col=0)

### Process

In [23]:
sentence_values = [str(i) for i in list(dataset['sentence'].values)]

In [18]:
sentence_embedding = model.encode(sentence_values)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 965/965 [01:19<00:00, 12.08it/s]


In [19]:
len(sentence_embedding[0])

768

In [38]:
dataset['sentence'][1]

str

In [36]:
dataset['sentence'][10]

nan

In [40]:
sts_clutser_embeddings = []

nan_seen = False

# Loop both embeddings and enter them in the new array
for i in range(dataset.shape[0]):
    
    if str(dataset['sentence'][i]) == 'nan':
        
        if not nan_seen:
            sts_clutser_embeddings.append([
                dataset['row'][i],
                dataset['column'][i],
                dataset['sentence'][i],
                np.array(sentence_embedding[i])
            ])
            nan_seen = True
        else:
            sts_clutser_embeddings.append([
                dataset['row'][i],
                dataset['column'][i],
                dataset['sentence'][i],
                []
            ])
    else:
        sts_clutser_embeddings.append([
            dataset['row'][i],
            dataset['column'][i],
            dataset['sentence'][i],
            np.array(sentence_embedding[i])
        ])

In [41]:
sts_clutser_embeddings[5:10]

[[0,
  5,
  'muss vor allem weiche oder flüssige Speisen zu mir nehmen',
  array([ 1.23842824e-02,  5.49952388e-02,  2.01824624e-02, -4.95984070e-02,
          5.06091416e-02,  4.77832817e-02, -2.48470847e-02, -3.85949798e-02,
         -1.20105809e-02,  5.92956133e-02, -1.61763709e-02,  4.93639596e-02,
         -9.94772092e-03,  2.78619248e-02, -3.87338027e-02,  2.11774614e-02,
         -6.95648044e-02,  4.53111157e-02, -7.28162453e-02,  2.41670888e-02,
          1.04964329e-02,  6.55331789e-03,  3.00123170e-02,  4.52369452e-02,
          7.23095685e-02,  7.64904767e-02, -4.35298961e-03, -4.36029676e-03,
         -2.04209965e-02,  2.03726590e-02,  2.47783810e-02,  4.14117575e-02,
          3.36179808e-02,  6.17429614e-02,  6.67658448e-03,  2.26166584e-02,
          1.31901773e-03, -1.79713275e-02, -6.31443635e-02, -2.64772587e-02,
         -1.56635642e-02, -5.86713590e-02,  1.67650729e-02,  4.87194508e-02,
         -3.83972144e-03,  2.97048092e-02, -3.52059416e-02, -1.17755029e-02,
   

In [42]:
sts_cluster_input = pd.DataFrame(sts_clutser_embeddings)
sts_cluster_input.columns = ['row','column','sentence','embedding']

In [43]:
sts_cluster_input

,row,column,sentence,embedding
0,0,0,Schmerzen im Kiefergelenk,"[0.03406202, 0.013992228, 0.014252492, -0.0304..."
1,0,1,Schmerzfreiheit,"[-0.07252122, 0.026934693, -0.04297495, 0.0319..."
2,0,2,"essen vor allem harte, zähe Speisen, wenn das ...","[-0.020250382, 0.08037725, 0.008891177, -0.054..."
3,0,3,Essen,"[-0.06400962, 0.014543581, 0.015365832, 0.0073..."
4,0,4,wenn der Kiefer entspannt ist,"[-0.00921524, 0.050368443, 0.02311897, 0.03054..."
...,...,...,...,...
61707,2127,24,NaN,[]
61708,2127,25,NaN,[]
61709,2127,26,NaN,[]
61710,2127,27,NaN,[]


In [33]:
# Check folder exists
Path('./data/output').mkdir(parents=True, exist_ok=True)

# Export pandas dataframe to .csv
sts_cluster_input.to_csv('data/output/dental_cluster_input.csv')